In [1]:
import os

from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_deepseek import ChatDeepSeek
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_text_splitters import CharacterTextSplitter
from pprint import pprint
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI

/home/hieu/miniconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vector embedding

In [3]:
with open("cleaned_data/format_data.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()

splitter = CharacterTextSplitter(
    separator="==================================================",
    chunk_size=1000,
    chunk_overlap=0,
    is_separator_regex=False
)

docs = splitter.create_documents([raw_text])
len(docs)

Created a chunk of size 2439, which is longer than the specified 1000
Created a chunk of size 1773, which is longer than the specified 1000
Created a chunk of size 1919, which is longer than the specified 1000
Created a chunk of size 2031, which is longer than the specified 1000
Created a chunk of size 2754, which is longer than the specified 1000
Created a chunk of size 2182, which is longer than the specified 1000
Created a chunk of size 2592, which is longer than the specified 1000
Created a chunk of size 4612, which is longer than the specified 1000
Created a chunk of size 1610, which is longer than the specified 1000
Created a chunk of size 1948, which is longer than the specified 1000
Created a chunk of size 1802, which is longer than the specified 1000
Created a chunk of size 1659, which is longer than the specified 1000
Created a chunk of size 2435, which is longer than the specified 1000
Created a chunk of size 2379, which is longer than the specified 1000
Created a chunk of s

1085

In [4]:
# embeddings = GoogleGenerativeAIEmbeddings(model="text-embedding-004")

# Cách này cho deploy
# # Khi chưa có db
# vector_store = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings,
#     persist_directory="vector_db"
# )

# Khi db đã có

# vector_store = Chroma(
#     persist_directory="vector_db",
#     embedding_function=embeddings
# )
#
# retriever = vector_store.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 3}
# )

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={"device": "cuda"}
)

vector_store = InMemoryVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# II. Chain

In [9]:
# model = ChatDeepSeek(
#     model="deepseek-chat",
#     temperature=0,
# )

# model = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     temperature=0,
# )

# model = ChatOpenAI(
#     model="GPT-4o mini",
#     temperature=0
# )

model = ChatOllama(
    model="qwen3:4b",
    reasoning=False,
    temperature=0
)

### 1. Rewrite question Chain

In [10]:
contextualize_q_system_prompt = """
Bạn là một chuyên gia tối ưu hóa truy vấn tìm kiếm cho hệ thống AI bán hàng.
Nhiệm vụ của bạn là viết lại câu hỏi mới nhất của người dùng thành một "Câu hỏi độc lập" (Standalone Question) để hệ thống có thể tìm kiếm dữ liệu chính xác nhất.

Dựa trên Lịch sử trò chuyện (Chat History) và Câu hỏi mới (Input):
1.  **Xác định ngữ cảnh:** Tìm xem người dùng đang nói về sản phẩm nào trong lịch sử chat.
2.  **Thay thế đại từ:** Thay thế các từ như "nó", "cái đó", "sản phẩm này", "màu đó" bằng TÊN SẢN PHẨM CỤ THỂ hoặc THUỘC TÍNH CỤ THỂ.
3.  **Loại bỏ từ thừa:** Bỏ các từ xã giao (vâng, dạ, shop ơi, alo...) để câu hỏi ngắn gọn, tập trung vào từ khóa.
4.  **Giữ nguyên:** Nếu câu hỏi đã đầy đủ chủ ngữ/vị ngữ hoặc đang hỏi vấn đề mới không liên quan lịch sử, hãy giữ nguyên.

QUY TẮC BẤT DI BẤT DỊCH:
- KHÔNG được trả lời câu hỏi.
- KHÔNG thêm thông tin bịa đặt.
- Chỉ trả về duy nhất nội dung câu hỏi đã viết lại.
"""

# Phần code LangChain giữ nguyên
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

rephrase_chain = contextualize_q_prompt | model | StrOutputParser()

### 2. Answer Chain

In [11]:
system_prompt = """## VAI TRÒ
Bạn là một Trợ lý Tư vấn Bán hàng AI chuyên nghiệp, nhiệt tình và am hiểu sản phẩm. Nhiệm vụ của bạn là giải đáp thắc mắc và tư vấn cho khách hàng dựa trên danh sách sản phẩm được cung cấp trong [CONTEXT].

## QUY TẮC CỐT LÕI (BẮT BUỘC TUÂN THỦ)
1.  **Dữ liệu duy nhất:** Chỉ được sử dụng thông tin có trong [CONTEXT] để trả lời. Tuyệt đối không tự bịa ra thông tin bên ngoài (như hạn sử dụng, thành phần).
2.  **Xử lý thiếu tin:** Nếu khách hỏi thông tin không có trong context, hãy trả lời: "Dạ hiện tại thông tin này nhà sản xuất chưa cập nhật chi tiết, bạn có thể tham khảo thêm ở link sản phẩm hoặc nhắn tin trực tiếp cho shop nhé ạ."
3.  **Hiển thị hình ảnh:** Khi giới thiệu sản phẩm, BẮT BUỘC phải hiển thị ảnh minh họa đầu tiên bằng cú pháp Markdown: `![Tên sản phẩm](Link_ảnh_đầu_tiên)`.
4.  **Không spam:** Nếu tìm thấy quá nhiều sản phẩm (trên 3), chỉ liệt kê tên và giá, sau đó hỏi khách muốn xem chi tiết món nào.

## ĐỊNH DẠNG CÂU TRẢ LỜI
Khi khách hàng hỏi về một sản phẩm cụ thể, hãy trình bày theo cấu trúc sau:

**[Tên sản phẩm in đậm]**
![Hình ảnh minh họa](link_ảnh_số_1_trong_danh_sách)
- **Giá bán:** [Giá] (Nhớ ghi chú nếu có Flash Sale)
- **Điểm nổi bật:** Tóm tắt 2-3 ý chính từ phần "Mô tả sản phẩm" hoặc "Thuộc tính" (ví dụ: công dụng, loại da phù hợp).
- **Tình trạng:** [Còn hàng/Hết hàng] - [Thời gian giao dự kiến]
- **Link mua ngay:** [Link sản phẩm]

## PHONG CÁCH GIAO TIẾP
- Xưng hô: "Em" hoặc "Shop" - gọi khách là "Bạn" hoặc "Anh/Chị".
- Giọng văn: Niềm nở, khơi gợi nhu cầu. Ví dụ: Thay vì nói "Đây là kem dưỡng", hãy nói "Dạ mẫu kem dưỡng này đang rất hot, giúp trắng da hiệu quả lắm ạ".
- Luôn kết thúc bằng một câu gợi mở: "Bạn cần shop tư vấn kỹ hơn về màu sắc hay cách dùng không ạ?"

## ĐẦU VÀO DỮ LIỆU
Dữ liệu sản phẩm sẽ được cung cấp dưới dạng văn bản có cấu trúc "=== SẢN PHẨM ===". Hãy phân tích kỹ các trường: Giá, Đánh giá, Mô tả, và Danh sách ảnh."""

prompt = ChatPromptTemplate([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history", optional=True),
    ("user", "{question}")
])

rag_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x["rephrase_question"])),
        "history": lambda x: x.get("history_chat", []),
        "question": lambda x: x["question"]
    }
    | prompt
    | model
    | StrOutputParser()
)

In [12]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

history_chat = []
while True:
    question = input("User: ")
    print("User: ", question, end="\n\n")
    if question.lower() == "exit":
        break

    rephrase_question = rephrase_chain.invoke({
        "question": question,
        "history": history_chat
    })

    answer = rag_chain.invoke({
        "rephrase_question": rephrase_question,
        "question": question,
        "history": history_chat
    })
    print("Answer: ", answer, end="\n\n")

    history_chat.append(HumanMessage(question))
    history_chat.append(AIMessage(answer))

User:  bạn là ai ?



KeyboardInterrupt: 